# Tutorial: Advanced Generative Adversarial Network Techniques

In [20]:
import numpy as np
import tensorflow as tf
from plotting import plot_images
import tutorial
layers = tf.layers
print("TensorFLow version", tf.__version__)

('TensorFLow version', '1.13.1')


Let's start to build a data pipeline.
First we need to define our Data generator.
The generator should output real samples (input for the discriminator) and noise (input for the generator)
The variable LATENT_DIM defines the dimensionality of the latent space of the generator.
(The noise distribution we sample from).

In [5]:
def generator(LATENT_DIM):
    while True:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        images = (np.expand_dims(x_train, axis=-1)) / 255.
        images = images.astype(np.float32)
        noise = np.random.randn(60000, LATENT_DIM).reshape(60000, LATENT_DIM)
        idx = np.random.permutation(60000)
        noise = noise[idx]
        images = images[idx]
        for i in range(60000):
            yield (noise[i], images[i])

Let us now check if our generator is working

In [6]:
import itertools
test_image = np.array(list(itertools.islice(generator(64), 1)))
test_image.shape

(1, 2)

To train our estimator we can make create a TensorflowDataset out of our data generator.
The function outputs a batches of our dataset.

In [7]:
def batch_dataset(BATCH_SIZE, LATENT_DIM, generator_fn):
    Dataset = tf.data.Dataset.from_generator(
        lambda: generator_fn(LATENT_DIM), output_types=(tf.float32, tf.float32),
        output_shapes=(tf.TensorShape((LATENT_DIM,)), tf.TensorShape((28, 28, 1))))
    return Dataset.batch(BATCH_SIZE)

For training a GAN we nee to further define our generator and discriminator network.
We start by defining our generator network, which should map from our noise space into the space of out images (LATENT_DIM --> IMAGE_DIM)

In [15]:
def generator_fn(x, latent_dim=LATENT_DIM):
    x = layers.Dense(7 * 7 * 128, activation='relu', input_shape=(latent_dim,))(x)  #
    x = tf.reshape(x, shape=[BATCH_SIZE, 7, 7, 128])
    x = layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.Conv2D(1, (5, 5), padding='same', activation='sigmoid')(x)
    return x

After defining our generator network we need now to implement our discriminator.
The task of the discriminator is to measure the similarity between the fake images (output of the generator) and the real images.
So, the network maps from the image space into a 1D space where we can measure the 'distance' between the distributions of the real and generated images.  (IMAGE_DIM --> 1)

In [9]:
def discriminator_fn(x, drop_rate=0.25):
    """ Discriminator network """
    x = layers.Conv2D(32, (5, 5), padding='same', strides=(2, 2), activation='relu', input_shape=(28, 28, 1))(x)
    x = tf.nn.leaky_relu(x, 0.2)
    x = layers.Conv2D(64, (5, 5), padding='same', strides=(2, 2), activation='relu')(x)
    x = tf.nn.leaky_relu(x, 0.2)
    x = layers.Conv2D(128, (5, 5), padding='same', strides=(2, 2), activation='relu')(x)
    x = tf.nn.leaky_relu(x, 0.2)
    x = layers.Flatten()(x)
    x = layers.Dense(256)(x)
    x = tf.nn.leaky_relu(x, 0.2)
    x = layers.Dense(1)(x)
    return x

As Wasserstein-1 is a meaningful distance measure for disjoint distributions let's use it as objective of our GAN training.
We can very easily make use of the losses predefined in tf.contrib.gan.
The are 2 possible constraints to construct the Wasserstein distance:
- Use weight clipping
- Penalize the gradient

(Easy interpretation: We need a constraint to train the discriminator to convergence, otherwise the discriminator could focus on one feature which differs between real and fake samples and won't converge)
Weight clamping will heavily reduce the capacity of the discriminator which is unfavourable.
So let use use the Gradient Penalty (https://arxiv.org/abs/1704.00028):
By penalizing the gradient to be smaller than 1, we enforce the lipschitz constraint needed to construct Wasserstein using the Kantorovich-Rubinstein duality(https://cedricvillani.org/wp-content/uploads/2012/08/preprint-1.pdf)

In [10]:
def discrimintator_loss(model, add_summaries=True):

    loss = tf.contrib.gan.losses.wasserstein_discriminator_loss(model, add_summaries=add_summaries)
    gp_loss = GP * tf.contrib.gan.losses.wasserstein_gradient_penalty(model, epsilon=1e-10, one_sided=True, add_summaries=add_summaries)
    loss += gp_loss

    if add_summaries:
        tf.summary.scalar('discriminator_loss', loss)

    return loss

After defining our loss we can choose our training parameters

In [11]:
BATCH_SIZE = 32  # number of samples fed into the framework in each iteration
LATENT_DIM = 64  # dimension of the generators latent space
GEN_LR = 0.001   # learning rate of the generator
DIS_LR = 0.0001  # learning rate of the discriminator
ITER = 1000      # framework iterations
LOG_DIR = "."    # directory of the estimator (to save the graph and checkpoints)
dir = tutorial.make_dir(LOG_DIR, "WGAN_GP")
GP = 10          # factor to scale the gradient penalty (higher means larger enforcing the Lipschitz constrain)
N_CRIT = 5       # number of critic iterations per generator iterations.

Now we can very easily implement our framework as estimator using tfgan.
This will heavily simplify our training procedure.

In [16]:
tfgan = tf.contrib.gan
gan_estimator = tfgan.estimator.GANEstimator(
    dir,
    generator_fn=generator_fn,
    discriminator_fn=discriminator_fn,
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss,
    discriminator_loss_fn=discrimintator_loss,
    generator_optimizer=tf.train.AdamOptimizer(GEN_LR, 0.5),
    discriminator_optimizer=tf.train.AdamOptimizer(DIS_LR, 0.5),
    get_hooks_fn=tfgan.get_sequential_train_hooks(tfgan.GANTrainSteps(1, N_CRIT)),
    config=tf.estimator.RunConfig(save_summary_steps=10, keep_checkpoint_max=1, save_checkpoints_steps=200),
    use_loss_summaries=True)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb89959890>, '_model_dir': './WGAN_GP_train_2019-04-04_17:03:06', '_protocol': None, '_save_checkpoints_steps': 200, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 10, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


Let us train our framework using our gan_estimator and our data_pipeline

In [ ]:
gan_estimator.train(lambda: batch_dataset(BATCH_SIZE, LATENT_DIM, generator), steps=ITER)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./WGAN_GP_train_2019-04-04_17:03:06/model.ckpt.
INFO:tensorflow:loss = 0.041437, step = 1
INFO:tensorflow:global_step/sec: 0.434456
INFO:tensorflow:loss = -5.485588, step = 101 (230.182 sec)
